In [6]:
# The C2QA pacakge is currently not published to PyPI. 
# To use the package locally, add the C2QA repository's root folder to the path prior to importing c2qa.
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Displacement Gate Calibration
The following circuit is a way to calibrate the conditional displacement gate. 

In [15]:
import c2qa
import numpy
import qiskit

qmr = c2qa.QumodeRegister(1, 2)
qr = qiskit.QuantumRegister(1)
cr = qiskit.ClassicalRegister(1)
circuit = c2qa.CVCircuit(qmr, qr, cr)

# qr[0] and cr[0] will init to zero
circuit.cv_initialize(0, qmr[0])

alpha = numpy.sqrt(numpy.pi)

circuit.h(qr[0])
circuit.cv_cnd_d(alpha, -alpha, qr[0], qmr[0])
circuit.cv_d(1j * alpha, qmr[0])
circuit.cv_cnd_d(-alpha, alpha, qr[0], qmr[0])
circuit.cv_d(-1j * alpha, qmr[0])
circuit.h(qr[0])
circuit.measure(qr[0], cr[0])

backend = qiskit.Aer.get_backend('statevector_simulator')
job = qiskit.execute(circuit, backend)
result = job.result()

state = result.get_statevector(circuit)
counts = result.get_counts(circuit)

print(circuit.draw("text"))
print(state)
print(counts)

qiskit.visualization.plot_state_city(state).savefig("state.png")

# Plotting the Wigner function has been removed from the main QisKit release.
#   See: https://qiskit.org/documentation/release_notes.html#visualization
#qiskit.visualization.plot_wigner_function(state).savefig("wigner.png")


         ┌──────────────────────┐┌─────┐┌────┐┌─────┐┌────┐   
q3263_0: ┤0                     ├┤1    ├┤0   ├┤1    ├┤0   ├───
         │  initialize(1,0,0,0) ││     ││  D ││     ││  D │   
q3263_1: ┤1                     ├┤2 Dc ├┤1   ├┤2 Dc ├┤1   ├───
         └────────┬───┬─────────┘│     │└────┘│     │├───┬┘┌─┐
q3264_0: ─────────┤ H ├──────────┤0    ├──────┤0    ├┤ H ├─┤M├
                  └───┘          └─────┘      └─────┘└───┘ └╥┘
  c11: 1/═══════════════════════════════════════════════════╩═
                                                            0 
[ 1.53535657e-01-4.27235371e-16j -1.10551735e-15-3.41272040e-02j
 -4.60246197e-01+5.34659352e-17j  1.41950724e-15-8.73748004e-01j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00-0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
{'0': 1}
